In [1]:
import pandas as pd

In [5]:
compras_df = pd.read_csv('data/clean_compras.csv')

In [6]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

In [2]:
## Empezaos con una expansion de datos que tenemos, fecha por ejemplo

In [10]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [11]:
compras_df['DIA'] = compras_df['FECHA'].dt.day

In [12]:
compras_df['MES'] = compras_df['FECHA'].dt.month

In [13]:
compras_df['YEAR'] = compras_df['FECHA'].dt.year

In [3]:
## Proyectar variables contra otras

In [16]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [17]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [21]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x : beneficiario_sales[x])

In [22]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [23]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [24]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x : sheet_mean_compras[x])

In [ ]:
## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [26]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('data/unique_beneficiarios.csv', index=None)

In [30]:
#CARGAR
linkedin_mty = pd.read_csv('data/linkedin_mty.csv')

In [32]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [34]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x : has_site[x.replace('  ',' ')])

In [35]:
compras_df['HAS_LINKEDIN_SITE']

0       True
1       True
2       True
3       True
4       True
        ... 
1535    True
1536    True
1537    True
1538    True
1539    True
Name: HAS_LINKEDIN_SITE, Length: 1540, dtype: bool

In [36]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

HAS_LINKEDIN_SITE
True     1392
False     148
Name: count, dtype: int64

In [37]:
compras_df.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO,HAS_LINKEDIN_SITE
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020,13,881659.731797,True
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020,13,881659.731797,True
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020,13,881659.731797,True
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS VARIOS,11,12,2020,13,881659.731797,True
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS VARIOS,23,12,2020,13,881659.731797,True


In [38]:
compras_df.to_csv('data/compras_df_enriched.csv', index=None)